In [1]:
from pprint import pprint
import json
import os
import requests
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin, SFType


In [2]:
username = 'saivarma844@gmail.com'
password = 'Sehwag@9831'
security_token = '26ckwUyEuCxXwfbCnzLESCmT'
domain = 'login' # or test

objects = ['Account','Opportunity']

session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
sf = Salesforce(instance=instance, session_id=session_id)

instance_name = sf.sf_instance
folder_path = r'C:\Users\Public\Documents'

In [3]:
def download_file(row,folder_path,i):
    record_id = row[1]['Id']
    file_name = row[1]['Id']+'_'+row[1]['PathOnClient']
    attachment_url = row[1]['VersionData']
    file_name=file_name.replace('/','_')

    if not os.path.exists(os.path.join('{}/{}'.format(folder_path,i))):
        os.mkdir(os.path.join('{}/{}'.format(folder_path,i)))       
        
    request = sf.session.get('https://{0}{1}'.format(instance_name, attachment_url), headers=sf.headers,stream=True)
    with open(os.path.join(folder_path,i,file_name), 'wb') as f:
        f.write(request.content)
        f.close()

In [12]:
for i in objects:
    query = 'select ContentDocumentId, Id,LinkedEntityId, ShareType, Visibility  from contentdocumentlink where LinkedEntityId in (Select Id  from {})'.format(i)
    response = sf.query(query)
    lstRecords = response.get('records')
    nextRecordsUrl = response.get('nextRecordsUrl')

    while not response.get('done'):
            response = sf.query_more(nextRecordsUrl, identifier_is_url=True)
            lstRecords.extend(response.get('records'))
            nextRecordsUrl = response.get('nextRecordsUrl')

    df_records = pd.DataFrame(lstRecords)
    if not os.path.exists(os.path.join('{}\{}'.format(folder_path,i))):
        os.mkdir(os.path.join('{}\{}'.format(folder_path,i))) 
    df_records.to_csv('{}\{}\ContentDocumentLink.csv'.format(folder_path,i))

    ac=list(df_records['ContentDocumentId'])
    querySOQL = """select Id,ContentDocumentId, FileExtension, FileType, IsLatest, IsMajorVersion, Launchbox_Id__c, PathOnClient, PublishStatus, PositiveRatingCount, ReasonForChange, SharingOption, SharingPrivacy, Title, VersionData, VersionNumber, NegativeRatingCount from contentversion """
    response = sf.query(querySOQL)
    lstRecords = response.get('records')
    nextRecordsUrl = response.get('nextRecordsUrl')

    while not response.get('done'):
        response = sf.query_more(nextRecordsUrl, identifier_is_url=True)
        lstRecords.extend(response.get('records'))
        nextRecordsUrl = response.get('nextRecordsUrl')

    df_records1 = pd.DataFrame(lstRecords)
    data=df_records1[df_records1['ContentDocumentId'].isin(ac)]
    success=pd.DataFrame()
    Failure=pd.DataFrame()
    for  row in data.iterrows():
        try:
            download_file(row,folder_path,i)
            success = success.append(row[1], ignore_index=True)
        except Exception as e:
            failure = failure.append(row[1], ignore_index=True)
        finally:
            continue
    data['target_path_on_Clinet'] = folder_path+'\\'+i+'\\'+data['Id']+'_'+data['PathOnClient']
    data['target_path_on_Clinet'] =data['target_path_on_Clinet'].replace('/','_')
    data['target_version_data']= data['target_path_on_Clinet']
    data.to_csv('{}\{}\ContentVersion.csv'.format(folder_path,i))
    
    

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5272\4210919186.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target_path_on_Clinet'] = folder_path+'\\'+i+'\\'+data['Id']+'_'+data['PathOnClient']
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5272\4210919186.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target_path_on_Clinet'] =data['target_path_on_Clinet'].replace('/','_')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5272\4210919186.py:42: SettingWithCopyWarning: 
A value is trying to 

In [13]:
data

,attributes,Id,ContentDocumentId,FileExtension,FileType,IsLatest,IsMajorVersion,Launchbox_Id__c,PathOnClient,PublishStatus,PositiveRatingCount,ReasonForChange,SharingOption,SharingPrivacy,Title,VersionData,VersionNumber,NegativeRatingCount,target_path_on_Clinet,target_version_data
6,"{'type': 'ContentVersion', 'url': '/services/d...",0685j00000MFpH1AAL,0695j00000MPVJcAAP,csv,CSV,True,True,None,each_ball_records.csv,P,0,None,A,N,each_ball_records,/services/data/v59.0/sobjects/ContentVersion/0...,1,0,C:\Users\Public\Documents\Opportunity\0685j000...,C:\Users\Public\Documents\Opportunity\0685j000...


In [ ]:
# Success Records 
success

In [ ]:
#Failure Records 
failure